# Exploring Project

#### library import

In [2]:
import sys
sys.path.append(r'.\src\utils')
sys.path.append(r'.\configure')

from sqlquerytools import SqlQueryResult
from configure import conf

# import libraries
import pyodbc
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from IPython.core.display_functions import display

#### Connection to the server

In [3]:
# Establishing connection to SQL database
print('*** Connecting to SQL Server... ***')
conn = pyodbc.connect(str(conf.configs['db']))
print('*** Connection Established. ***')

*** Connecting to SQL Server... ***
*** Connection Established. ***


#### Query the databases

In [3]:
# SQL Query that will be sent, response is the table of interest
query_str = '''
select b.BetID,
b.ClientID,
b.BetDateTime,
b.TotalStake,
LCategoryClass.CategoryClass


from dbo.Bet as b

left join dbo.Client as c on b.ClientID=c.ClientID
left join dbo.LVerification as verify on c.VerificationID=verify.VerificationID

left join dbo.BetDetail on dbo.BetDetail.BetID=b.BetID
left join dbo.[Event] on dbo.BetDetail.EventID=dbo.[Event].EventID

left join dbo.LEventClass on dbo.[Event].EventClassID=dbo.LEventClass.EventClassID
left join dbo.LMasterEventClass on dbo.LEventClass.MasterEventClassID=dbo.LMasterEventClass.MasterEventClassID
left join dbo.LCategoryClass on dbo.LMasterEventClass.CategoryClassID=dbo.LCategoryClass.CategoryClassID

where c.IsTest=0 and c.IsBookmaker=0 and verify.IsVerified=1 and b.FreeBetID=0 and dbo.[Event].EventTypeID=1 and dbo.LCategoryClass.CategoryClass LIKE '%NSW%';
'''

In [4]:
# Send the SQL Query, and record response as a Pandas dataframe.
sql_query = pd.read_sql_query(query_str, conn)
df_raw = pd.DataFrame(sql_query)

pd.set_option("display.max_columns", 8)
pd.set_option("display.max_rows", 30)
display(df_raw)

c:\users\zhaowang\onedrive - irpsx pty limited\desktop\bettube_project_0\venv\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,BetID,ClientID,BetDateTime,TotalStake,CategoryClass
0,22895,6131,2016-09-30 01:19:56.680,50.0,NSW Metro Thoroughbred
1,23414,6131,2016-09-30 04:39:24.457,50.0,NSW Country Thoroughbred
2,25168,6131,2016-09-30 11:07:47.940,20.0,NSW Metro Thoroughbred
3,25249,6131,2016-09-30 11:19:52.720,20.0,NSW Metro Thoroughbred
4,28912,6131,2016-10-01 05:58:47.560,200.0,NSW Metro Thoroughbred
...,...,...,...,...,...
1415848,5403612,58763,2019-09-20 06:00:23.713,7.0,NSW Country Thoroughbred
1415849,5363509,58804,2019-09-14 02:09:06.937,50.0,NSW Metro Thoroughbred
1415850,5403901,58820,2019-09-20 06:27:24.090,25.0,NSW Country Thoroughbred
1415851,5410837,58829,2019-09-21 03:13:31.317,5.0,NSW Country Thoroughbred


In [5]:
df_raw.head(100)

,BetID,ClientID,BetDateTime,TotalStake,CategoryClass
0,22895,6131,2016-09-30 01:19:56.680,50.0,NSW Metro Thoroughbred
1,23414,6131,2016-09-30 04:39:24.457,50.0,NSW Country Thoroughbred
2,25168,6131,2016-09-30 11:07:47.940,20.0,NSW Metro Thoroughbred
3,25249,6131,2016-09-30 11:19:52.720,20.0,NSW Metro Thoroughbred
4,28912,6131,2016-10-01 05:58:47.560,200.0,NSW Metro Thoroughbred
...,...,...,...,...,...
95,1166230,6131,2017-09-06 05:06:13.190,50.0,NSW Metro Thoroughbred
96,1168981,6131,2017-09-07 02:58:52.480,50.0,NSW Country Thoroughbred
97,1169483,6131,2017-09-07 04:35:25.017,50.0,NSW Country Thoroughbred
98,1169831,6131,2017-09-07 05:23:37.757,50.0,NSW Country Thoroughbred


In [6]:
df = df_raw.copy()

# Create a BetYear column
df.insert(3, "BetYear", df['BetDateTime'].dt.year, True)

# Remove duplicates due to multi's/left-joining(?)
df = df.drop_duplicates(['ClientID', 'BetDateTime', 'BetID', 'TotalStake'])

# Remove stakes with non-positive values
df = df[df["TotalStake"] > 0]

display(df)

,BetID,ClientID,BetDateTime,BetYear,TotalStake,CategoryClass
0,22895,6131,2016-09-30 01:19:56.680,2016,50.0,NSW Metro Thoroughbred
1,23414,6131,2016-09-30 04:39:24.457,2016,50.0,NSW Country Thoroughbred
2,25168,6131,2016-09-30 11:07:47.940,2016,20.0,NSW Metro Thoroughbred
3,25249,6131,2016-09-30 11:19:52.720,2016,20.0,NSW Metro Thoroughbred
4,28912,6131,2016-10-01 05:58:47.560,2016,200.0,NSW Metro Thoroughbred
...,...,...,...,...,...,...
1415848,5403612,58763,2019-09-20 06:00:23.713,2019,7.0,NSW Country Thoroughbred
1415849,5363509,58804,2019-09-14 02:09:06.937,2019,50.0,NSW Metro Thoroughbred
1415850,5403901,58820,2019-09-20 06:27:24.090,2019,25.0,NSW Country Thoroughbred
1415851,5410837,58829,2019-09-21 03:13:31.317,2019,5.0,NSW Country Thoroughbred


In [7]:
# Determine the TotalStake over 2019

df_2019 = df.copy()
# take only bets in 2019
df_2019 = df_2019[df_2019["BetYear"] == 2019]

# Sum up the total stake amounts
totalStaked = df_2019["TotalStake"].sum()
print("NSW Thoroughbred (Racing) Turnover (2019): $", round(totalStaked, 2))
print("Estimated (25%): $", round(totalStaked*0.25, 2))


# Total staked by year
df_temp = df.copy().groupby(["BetYear"])["TotalStake"].sum()
display(df_temp)

NSW Thoroughbred (Racing) Turnover (2019): $ 25552310.12
Estimated (25%): $ 6388077.53


BetYear
2016    6.883862e+06
2017    3.507139e+07
2018    2.486363e+07
2019    2.555231e+07
2020    1.462236e+07
Name: TotalStake, dtype: float64

## Explore the database

In [48]:
query = '''
SELECT TOP(100) *
FROM CLIENT AS C WITH (NOLOCK)
WHERE C.ARCHIVEPARTITION IN (0,1)
OPTION (MAXDOP 1)

'''


# Send the SQL Query, and record response as a Pandas dataframe.
test_sql_query = pd.read_sql_query(query, conn)
df_raw_test = pd.DataFrame(test_sql_query)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df_raw_test.head(100)

df_raw_test.info(True,null_counts=True)

c:\users\zhaowang\onedrive - irpsx pty limited\desktop\bettube_project_0\venv\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\ZhaoWang\AppData\Local\Temp\ipykernel_17532\3327387670.py:18: FutureWarning: null_counts is deprecated. Use show_counts instead
  df_raw_test.info(True,null_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 36 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   ArchivePartition             100 non-null    int64         
 1   ClientID                     100 non-null    int64         
 2   OrgID                        100 non-null    int64         
 3   AccountNumber                100 non-null    object        
 4   Gender                       100 non-null    object        
 5   SignupDate                   100 non-null    datetime64[ns]
 6   SignupChannelID              100 non-null    int64         
 7   IsCreditClient               100 non-null    bool          
 8   ClientProfileID              100 non-null    int64         
 9   BetInterceptRuleID           100 non-null    int64         
 10  IsTest                       100 non-null    bool          
 11  IsBookmaker                  100 non-null    b

In [55]:
# Read the excel.
file_name = r'.\data\test.xlsx'
ana = pd.read_excel(file_name, header=0)
display(ana)

# rename the column names
ana.rename(columns={'CLIENTID':'ClientID'},inplace=True)

# mutil select dataset columns.
info_data = df_raw_test[['ClientID','EmailDomain','DOBMonth', 'DOBYear']]

,CLIENTID,Active,First Name,Last Name,EMAIL,Mobile,SURVEY_NAME,QUESTION,ANSWER,SUBMISSION_TIME
0,10,NaN,NaN,NaN,NaN,NaN,RPX survey,"Just quickly, where did you first hear about us?",through a friend,2021-11-24 23:05:56
1,18,NaN,NaN,NaN,NaN,NaN,LOOT survey,"Just quickly, where did you first hear about us?",through a friend,2021-11-28 02:26:24
2,20,NaN,NaN,NaN,NaN,NaN,Account open survey,"Noted, Thank You â˜º Also, can we ask how you...",through a friend,2022-01-25 17:39:02
3,6127,NaN,NaN,NaN,NaN,NaN,Melbourne Cup Questionnaire 03.11.2021,"Thank you! Now first things first, we'd love t...",through a friend ðŸ‘¯â€â™‚ï¸,2021-11-03 00:32:04


In [56]:
# Merge the info to fill the blank
q1 = pd.merge(info_data, ana, on=['ClientID'])

display(q1)



,ClientID,EmailDomain,DOBMonth,DOBYear,Active,First Name,Last Name,EMAIL,Mobile,SURVEY_NAME,QUESTION,ANSWER,SUBMISSION_TIME
0,10,dfsa.com,1,1980,NaN,NaN,NaN,NaN,NaN,RPX survey,"Just quickly, where did you first hear about us?",through a friend,2021-11-24 23:05:56
1,18,totalbettingsolutions.com.au,7,1975,NaN,NaN,NaN,NaN,NaN,LOOT survey,"Just quickly, where did you first hear about us?",through a friend,2021-11-28 02:26:24
2,20,gmail.com1,4,1988,NaN,NaN,NaN,NaN,NaN,Account open survey,"Noted, Thank You â˜º Also, can we ask how you...",through a friend,2022-01-25 17:39:02
3,6127,hotmail.com,2,1970,NaN,NaN,NaN,NaN,NaN,Melbourne Cup Questionnaire 03.11.2021,"Thank you! Now first things first, we'd love t...",through a friend ðŸ‘¯â€â™‚ï¸,2021-11-03 00:32:04


In [72]:
# info searching

query = '''
SELECT
    C.CLIENTID,
    C.ResidentialAddress1,
    C.ResidentialAddress2,
    C.ResidentialSuburb,
    C.ResidentialStateCode,
    C.DOBYear,
    C.DOBMonth,
    C.EmailDomain
FROM CLIENT AS C WITH (NOLOCK)
WHERE C.ARCHIVEPARTITION IN (0,1)
    AND ResidentialStateCode = 'NSW'
    AND ResidentialSuburb = 'CENTENNIAL PARK'
    AND DOBYEAR = 1975

OPTION (MAXDOP 1)

'''


# Send the SQL Query, and record response as a Pandas dataframe.
test_sql_query = pd.read_sql_query(query, conn)
df_raw_test = pd.DataFrame(test_sql_query)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df_raw_test.head(100)

c:\users\zhaowang\onedrive - irpsx pty limited\desktop\bettube_project_0\venv\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,CLIENTID,ResidentialAddress1,ResidentialAddress2,ResidentialSuburb,ResidentialStateCode,DOBYear,DOBMonth,EmailDomain
0,26241,cook,None,centennial park,NSW,1975,1,gmail.com
1,68084,ROBERTSON RD,None,CENTENNIAL PARK,NSW,1975,2,hotmail.com


In [84]:
# info searching

query = '''
SELECT *
FROM RClientAccountRule AS C WITH (NOLOCK)
JOIN LAccountRule AS LA WITH (NOLOCK) ON C.AccountRuleID = LA.AccountRuleID
WHERE C.ARCHIVEPARTITION IN (0,1)
    AND CLIENTID = 68084
OPTION (MAXDOP 1)

'''


# Send the SQL Query, and record response as a Pandas dataframe.
test_sql_query = pd.read_sql_query(query, conn)
df_raw_test = pd.DataFrame(test_sql_query)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df_raw_test.head(100)

c:\users\zhaowang\onedrive - irpsx pty limited\desktop\bettube_project_0\venv\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,ArchivePartition,ClientID,AccountRuleID,AccountRuleID,AccountRule,RuleType
0,0,68084,5,5,Disable Cashout,A
1,0,68084,6,6,Disable Multis,A
2,0,68084,101,101,No Marketing Emails,M
3,0,68084,102,102,No Marketing Calls,M
4,0,68084,889,889,No Marketing SMS,M
5,0,68084,890,890,No Bonuses,M


In [87]:
# info searching

query = '''
SELECT TOP(10) *
FROM LAccountRule AS C WITH (NOLOCK)
OPTION (MAXDOP 1)

'''


# Send the SQL Query, and record response as a Pandas dataframe.
test_sql_query = pd.read_sql_query(query, conn)
df_raw_test = pd.DataFrame(test_sql_query)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df_raw_test.head(100)


# print(df_raw_test.head(100))

c:\users\zhaowang\onedrive - irpsx pty limited\desktop\bettube_project_0\venv\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,AccountRuleID,AccountRule,RuleType
0,1,Disable Betting,A
1,2,Disable Withdrawal,A
2,3,Disable Deposit,A
3,4,Unverified - Disable Betting,A
4,5,Disable Cashout,A
5,6,Disable Multis,A
6,7,Unverified - Disable Deposit,A
7,8,Remove Cancel Withdrawal Option,A
8,101,No Marketing Emails,M
9,102,No Marketing Calls,M


#### SQL query library

In [4]:
# Testing the SQL query library

# Create the instance and initialization.
zhao_sql = SqlQueryResult()

*** Connecting to SQL Server... ***
*** Connection Established. ***


In [5]:
# without query there will be no result
zhao_sql.result()

Exception: Error: No Query. Use SqlQueryResult.query(code) or SqlQueryResult.result(code)

In [6]:
sql_code = '''
SELECT *
FROM RClientAccountRule AS C WITH (NOLOCK)
JOIN LAccountRule AS LA WITH (NOLOCK) ON C.AccountRuleID = LA.AccountRuleID
WHERE C.ARCHIVEPARTITION IN (0,1)
    AND CLIENTID = 68084
OPTION (MAXDOP 1)
'''

In [7]:
zhao_sql.query(sql_code)

c:\users\zhaowang\onedrive - irpsx pty limited\desktop\bettube_project_0\venv\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,ArchivePartition,ClientID,AccountRuleID,AccountRuleID,AccountRule,RuleType
0,0,68084,5,5,Disable Cashout,A
1,0,68084,6,6,Disable Multis,A
2,0,68084,101,101,No Marketing Emails,M
3,0,68084,102,102,No Marketing Calls,M
4,0,68084,889,889,No Marketing SMS,M
5,0,68084,890,890,No Bonuses,M


In [8]:
# export last query result
q1 = zhao_sql.result()
display(q1)

,ArchivePartition,ClientID,AccountRuleID,AccountRuleID,AccountRule,RuleType
0,0,68084,5,5,Disable Cashout,A
1,0,68084,6,6,Disable Multis,A
2,0,68084,101,101,No Marketing Emails,M
3,0,68084,102,102,No Marketing Calls,M
4,0,68084,889,889,No Marketing SMS,M
5,0,68084,890,890,No Bonuses,M


In [9]:
# direct export query result
sql_code2 = '''
SELECT TOP(10) *
FROM LAccountRule AS C WITH (NOLOCK)
OPTION (MAXDOP 1)
'''

q2 = zhao_sql.result(sql_code2)
display(q2)

c:\users\zhaowang\onedrive - irpsx pty limited\desktop\bettube_project_0\venv\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,AccountRuleID,AccountRule,RuleType
0,1,Disable Betting,A
1,2,Disable Withdrawal,A
2,3,Disable Deposit,A
3,4,Unverified - Disable Betting,A
4,5,Disable Cashout,A
5,6,Disable Multis,A
6,7,Unverified - Disable Deposit,A
7,8,Remove Cancel Withdrawal Option,A
8,101,No Marketing Emails,M
9,102,No Marketing Calls,M


In [10]:
# disconnection
zhao_sql.disconnect()

*** Connection Closed. ***


In [11]:
# reconnect
zhao_sql.connect_to_server()

*** Connecting to SQL Server... ***
*** Connection Established. ***


In [17]:
# zhao_sql.disconnect()
# zhao_sql.query(sql_code)
zhao_sql.connect_to_server()
# q2 = zhao_sql.result(sql_code2)



*** Connection Has Already Been Established. ***
